In [38]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

In [39]:
import pandas as pd
import numpy as np


In [40]:
df = pd.read_csv('../data/df1.csv').drop('Unnamed: 0', axis=1)
df

,Date,Return,Log Return,Volatility,Distance_MA50,Distance_EMA20,Cumulated_Return_5d,RSI14,Trend
0,2010-01-04 00:00:00-05:00,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,Bear
1,2010-01-05 00:00:00-05:00,0.002647,0.002644,NaN,NaN,0.001256,NaN,NaN,Bear
2,2010-01-06 00:00:00-05:00,0.000704,0.000703,NaN,NaN,0.001240,NaN,NaN,Bear
3,2010-01-07 00:00:00-05:00,0.004222,0.004213,NaN,NaN,0.003882,NaN,NaN,Bear
4,2010-01-08 00:00:00-05:00,0.003328,0.003322,NaN,NaN,0.005466,NaN,NaN,Bear
...,...,...,...,...,...,...,...,...,...
3768,2024-12-23 00:00:00-05:00,0.005988,0.005970,0.008463,0.008392,-0.002118,-0.016642,43.149037,Range
3769,2024-12-24 00:00:00-05:00,0.011115,0.011054,0.008789,0.018946,0.008112,-0.001599,46.098031,Range
3770,2024-12-26 00:00:00-05:00,0.000067,0.000067,0.008719,0.018203,0.007394,0.029140,46.981964,Range
3771,2024-12-27 00:00:00-05:00,-0.010526,-0.010582,0.009024,0.006985,-0.002905,0.018619,40.972868,Range


In [41]:
df1 = df.copy()
df1['Trend'] = df1['Trend'].map(lambda x: 0 if x == 'Range' else 1 if x == 'Bear' else 2)


days = np.arange(1, 3774, 1)
df1['Date'] = days
df1.head()

,Date,Return,Log Return,Volatility,Distance_MA50,Distance_EMA20,Cumulated_Return_5d,RSI14,Trend
0,1,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,1
1,2,0.002647,0.002644,NaN,NaN,0.001256,NaN,NaN,1
2,3,0.000704,0.000703,NaN,NaN,0.001240,NaN,NaN,1
3,4,0.004222,0.004213,NaN,NaN,0.003882,NaN,NaN,1
4,5,0.003328,0.003322,NaN,NaN,0.005466,NaN,NaN,1


In [42]:
from sklearn.impute import KNNImputer

#handles missing values et rename the columns accordingly
KNNI = KNNImputer()
new = KNNI.fit_transform(df1)
df1 = pd.DataFrame(new)

df1.columns = [
    "Date", "Return", "Log Return",
    "Volatility", "Distance_MA50", "Distance_EMA20",
    "Cumulated_Return_5d", "RSI14", "Trend"
]
df1

,Date,Return,Log Return,Volatility,Distance_MA50,Distance_EMA20,Cumulated_Return_5d,RSI14,Trend
0,1.0,0.002459,0.002455,0.011635,0.045560,0.000000,0.003797,32.310590,1.0
1,2.0,0.002647,0.002644,0.011635,0.045560,0.001256,0.003797,32.310590,1.0
2,3.0,0.000704,0.000703,0.011635,0.045560,0.001240,0.003797,32.310590,1.0
3,4.0,0.004222,0.004213,0.011635,0.045560,0.003882,0.003797,32.310590,1.0
4,5.0,0.003328,0.003322,0.011635,0.045560,0.005466,0.003797,32.310590,1.0
...,...,...,...,...,...,...,...,...,...
3768,3769.0,0.005988,0.005970,0.008463,0.008392,-0.002118,-0.016642,43.149037,0.0
3769,3770.0,0.011115,0.011054,0.008789,0.018946,0.008112,-0.001599,46.098031,0.0
3770,3771.0,0.000067,0.000067,0.008719,0.018203,0.007394,0.029140,46.981964,0.0
3771,3772.0,-0.010526,-0.010582,0.009024,0.006985,-0.002905,0.018619,40.972868,0.0


In [43]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [66]:

df_ts = df.copy()
df_ts['Trend'] = df_ts['Trend'].map(lambda x: 0 if x == 'Range' else 1 if x == 'Bear' else 2)


days = np.arange(1, 3774, 1)
df_ts['Date'] = days
df_ts = df_ts.dropna()

#train test split 0.8 respecting temporality

X_timed = df_ts.drop(['Trend', 'Date'], axis=1)
y_timed = df_ts['Trend']

X_train_timed = X_timed[:3017]
X_test_timed = X_timed[3017:]
y_train_timed = y_timed[:3017]
y_test_timed = y_timed[3017:]

print(len(X_train_timed) == len(y_train_timed))
print(len(X_test_timed) == len(y_test_timed))
X_train_timed

True
True


,Return,Log Return,Volatility,Distance_MA50,Distance_EMA20,Cumulated_Return_5d,RSI14
49,0.007966,0.007934,0.005395,0.043205,0.028045,0.017037,97.453454
50,0.005927,0.005910,0.005422,0.048680,0.030767,0.018527,100.000000
51,-0.000512,-0.000512,0.005433,0.047503,0.027264,0.013772,99.065484
52,-0.005062,-0.005075,0.005704,0.041690,0.019910,0.008554,88.870676
53,0.005346,0.005332,0.005718,0.046720,0.022881,0.013682,89.433988
...,...,...,...,...,...,...,...
3061,0.018396,0.018229,0.014713,-0.032596,-0.003256,0.037777,38.051796
3062,-0.004978,-0.004991,0.014484,-0.036592,-0.007442,0.017301,41.068372
3063,-0.008125,-0.008158,0.013686,-0.043112,-0.014050,-0.012747,44.151254
3064,-0.029479,-0.029922,0.014941,-0.069331,-0.039170,-0.039393,38.079570


### Baseline

In [67]:
from collections import Counter
print("Distribution des classes:")
print(Counter(y_test_timed))

# Baseline
baseline = max(Counter(y_test_timed).values()) / len(y_test_timed)
print(f"Baseline accuracy: {baseline:.2%}")

Distribution des classes:
Counter({0: 313, 2: 210, 1: 184})
Baseline accuracy: 44.27%


### Gaussian Naive Bayes model



In [68]:
NB1 = GaussianNB()
NB1.fit(X_train_timed, y_train_timed)

y_pred_NB = NB1.predict(X_test_timed)

conf_matrix = confusion_matrix(y_test_timed, y_pred_NB)
f1 = f1_score(y_test_timed, y_pred_NB, average = 'macro')

accuracy = accuracy_score(y_test_timed, y_pred_NB)

print(f'Confusion matrix for NB1: {conf_matrix}')
print(f'f1 score for NB1: {f1}')
print(f'Accuracy score for NB1: {accuracy}')

Confusion matrix for NB1: [[257   6  50]
 [127   5  52]
 [135   6  69]]
f1 score for NB1: 0.3432481432430016
Accuracy score for NB1: 0.4681753889674682


### Results

The metrics this model gives highlights that it is unable to capture the differences between the three regimes we try to predict. Another model may be able to be more precise adnd accurate based on the features the dataset has. 


### Random Forest Classifier

In [69]:
from sklearn.ensemble import RandomForestClassifier

In [70]:


KNNI2 = KNNImputer()
KNNI2.fit(X_train_timed)
train_Imp = KNNI2.transform(X_train_timed)
test_Imp = KNNI2.transform(X_test_timed)



In [71]:
scaler_timed = StandardScaler()

scaler_timed.fit(train_Imp)
X_train_timed_scaled = scaler_timed.transform(train_Imp)
X_test_timed_scaled = scaler_timed.transform(test_Imp)

X_train_timed_scaled = pd.DataFrame(X_train_timed_scaled)
X_test_timed_scaled = pd.DataFrame(X_test_timed_scaled)


X_train_timed_scaled.columns = [
    "Return", "Log Return",
    "Volatility", "Distance_MA50", "Distance_EMA20",
    "Cumulated_Return_5d", "RSI14"
]
X_test_timed_scaled.columns = [
    "Return", "Log Return",
    "Volatility", "Distance_MA50", "Distance_EMA20",
    "Cumulated_Return_5d", "RSI14"
]


In [72]:
RFC2 = RandomForestClassifier(n_estimators= 100, 
                              min_samples_leaf=1, 
                              max_features= 0.4, 
                              random_state=0)


RFC2.fit(X_train_timed_scaled, y_train_timed)

y_pred = RFC2.predict(X_test_timed_scaled)

conf_matrix_timed = confusion_matrix(y_test_timed, y_pred)
f1_timed = f1_score(y_test_timed, y_pred, average = 'macro')

accuracy_timed = accuracy_score(y_test_timed, y_pred)

print(f'Confusion matrix for RFC1: {conf_matrix_timed}')
print(f'f1 score for RFC1: {f1_timed}')
print(f'Accuracy score for RFC1: {accuracy_timed}')

Confusion matrix for RFC1: [[258  14  41]
 [127   8  49]
 [139   5  66]]
f1 score for RFC1: 0.35099085892943555
Accuracy score for RFC1: 0.4695898161244696


When evaluated under a strictly time-ordered split, model performance
drops substantially, highlighting the impact of temporal leakage in
random splits and confirming the limited out-of-sample predictability
of market regimes.


### End of V0

Under a strictly time-ordered evaluation protocol, none of the tested models
were able to extract stable and generalizable patterns from the available
technical indicators. This suggests that, in its current formulation,
short-horizon multi-class regime classification is intrinsically difficult,
likely due to the weak and non-stationary nature of the underlying signal.